In [1]:
import google.auth

In [2]:
import sys
import numpy as np

import scipy
from scipy.sparse import hstack
import pandas as pd
from tqdm import tqdm
import re

from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import GroupShuffleSplit, train_test_split
from sklearn.pipeline import Pipeline
from sklearn import preprocessing

In [3]:
from pecos.xmc.xlinear.model import XLinearModel
from pecos.xmc import Indexer, LabelEmbeddingFactory
from pecos.utils import smat_util

from pecos.utils import logging_util
logging_util.setup_logging_config(level=1) # LOGGING CONFIG

In [4]:
import nltk
nltk.download('punkt')
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Tf-idf vectorizer class

In [5]:
stemmer = SnowballStemmer(language='english')

simple_tokenizer = lambda t: re.findall('\w+', t)

# Replace all punctuation with spaces for 
# character-level tf-idf
def character_preprocessor(text):
    clear_text = re.sub('\W+', ' ', text)
    if clear_text[-1] != ' ':
        clear_text = clear_text + ' '
    if clear_text[0] != ' ':
        clear_text = ' ' + clear_text
    return clear_text


# Tf-idf vectorizer with:
#   - unigram word tf-idf
#   - bigram word tf-idf
#   - thrigram character tf-idf
# Concatecated together
class MultipleVectorizer:
    
    def __init__(self,
                 tokenizer,
                 lowercase=True,
                 stop_words=None,
                 min_df=1,
                 max_df=1.0,
                 max_features=None,
                 do_stemm=False):

        # Create vectorizers
        word_unigram_tfidf_vect = TfidfVectorizer(
            analyzer='word',
            ngram_range=(1, 1),
            tokenizer=tokenizer,
            lowercase=lowercase,
            min_df=min_df,
            max_df=max_df,
            max_features=max_features
        )

        word_bigram_tfidf_vect = TfidfVectorizer(
            analyzer='word',
            ngram_range=(2, 2),
            tokenizer=tokenizer,
            lowercase=lowercase,
            min_df=min_df,
            max_df=max_df,
            max_features=max_features
        )

        char_trigram_tfidf_vect = TfidfVectorizer(
            analyzer='char_wb',
            ngram_range=(3, 3),
            lowercase=lowercase,
            preprocessor=character_preprocessor,
            min_df=min_df,
            max_df=max_df,
            max_features=max_features
        )
        
        self.vectorizers = [word_unigram_tfidf_vect, 
                            word_bigram_tfidf_vect, 
                            char_trigram_tfidf_vect]
        
        self.do_stemm = do_stemm
        self.tokenizer = tokenizer

    

    
    def simple_stemmer(self, texts_list):
        result = []
        print('Stemming:')
        for text in tqdm(texts_list):
            words=self.tokenizer(text)
            words=[stemmer.stem(word) for word in words]
            stemmed_text = ' '.join(words)
            result.append(stemmed_text)
        return result


    def fit(self, X):

        if self.do_stemm:
            X = self.simple_stemmer(X)

        print("Fitting vectorizers:")
        for vectorizer in tqdm(self.vectorizers):
            vectorizer.fit(X)
        
    def transform(self, X):
        
        if self.do_stemm:
            X = self.simple_stemmer(X)

        print('Transforming:')
        output = []
        for vectorizer in tqdm(self.vectorizers):
            output.append(vectorizer.transform(X))
            
        stack = hstack(output)
        csr_matrix = scipy.sparse.csr_matrix(stack, dtype=np.float32)

        return csr_matrix
    
    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)
    

    def get_feature_names_out(self):
        output = []
        for vectorizer in self.vectorizers:
            output.append(vectorizer.get_feature_names_out())
        return np.hstack(output)

In [6]:
# Test tf-idf vectorizer

# Create vectorizer from multiple vectorizers
multiple_vect = MultipleVectorizer(
  lowercase=True,
  min_df=1,
  max_df=1.0,
  max_features=None,
  tokenizer=simple_tokenizer,
  do_stemm=False
)

X = ['''ab,    abb ab!!... ba baba 
aba aba abab! bab bab
  abab ab ab ab ab ab abb b bab ab ab ab ab ab''',
     'ab ab, abab ab, ab: abab ba baba baba ab aba. bab aba',
     'a a bab ab   abab ab ab abab, ab ab ab ab abab ab aba b',
     'ab, ab ab ab, ab ab ab   ba bab ab a, bab ab! ab ab ab a ba']
result = multiple_vect.fit_transform(X)
multiple_vect.get_feature_names_out()

Fitting vectorizers:


  0%|          | 0/3 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"
100%|██████████| 3/3 [00:00<00:00, 606.08it/s]


Transforming:


100%|██████████| 3/3 [00:00<00:00, 854.06it/s]


array(['a', 'ab', 'aba', 'abab', 'abb', 'b', 'ba', 'bab', 'baba', 'a a',
       'a ba', 'a bab', 'ab a', 'ab ab', 'ab aba', 'ab abab', 'ab abb',
       'ab ba', 'aba aba', 'aba abab', 'aba b', 'aba bab', 'abab ab',
       'abab ba', 'abab bab', 'abb ab', 'abb b', 'b bab', 'ba bab',
       'ba baba', 'bab ab', 'bab aba', 'bab abab', 'bab bab', 'baba ab',
       'baba aba', 'baba baba', ' a ', ' ab', ' b ', ' ba', 'ab ', 'aba',
       'abb', 'ba ', 'bab', 'bb '], dtype=object)

### Load dataset

In [7]:
dataset_filename = None
try:
    credentials, project = google.auth.default()
    dataset_filename = dataset_file_name # GCP
except:
    if 'google.colab' in sys.modules:
        dataset_filename = '/content/drive/MyDrive/qa_ncbi_dataset.tsv' # google colab
    else:
        dataset_filename = 'data/qa_ncbi_dataset.tsv' # local run
        
df = pd.read_csv(dataset_filename, sep='\t')
df.shape

(29319, 15)

In [8]:
df.question_id.nunique()

4040

### Split into dataframes

In [9]:
# Paper DF # Z
papers_df = df[['snippet_url', 'title', 'abstract']] \
    .drop_duplicates() \
    .reset_index(drop=True).reset_index() \
    .rename(columns={'index': 'paper_id'})
papers_df['title_abstract'] = papers_df['title'] + '. ' + papers_df['abstract']

df = df.merge(papers_df[['paper_id', 'snippet_url']])
papers_df = papers_df.set_index('paper_id')

# # Create paper id from url
# df['paper_id'] = df.snippet_id.str.split('_')
# df['paper_id'] = df['paper_id'].apply(lambda a: a[0]).astype(int)
papers_df.sample()

,snippet_url,title,abstract,title_abstract
paper_id,,,,
19942,http://www.ncbi.nlm.nih.gov/pubmed/25218447,Uncovering global SUMOylation signaling networ...,SUMOylation is a reversible post-translational...,Uncovering global SUMOylation signaling networ...


In [10]:
# question-paper matrix # Y
question_paper_df = pd.crosstab(df['question_id'], df['paper_id']).astype(int)
question_paper_df = question_paper_df[sorted(question_paper_df.columns)]
question_paper_df.sample()

paper_id,0,1,2,3,4,5,6,7,8,9,...,27794,27795,27796,27797,27798,27799,27800,27801,27802,27803
question_id,,,,,,,,,,,,,,,,,,,,,
533aaab6d6d3ac6a34000062,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Question df # X
questions_df = df[['question_id', 'question_text']] \
    .drop_duplicates('question_id') \
    .set_index('question_id').loc[question_paper_df.index]
questions_df.sample()

,question_text
question_id,
56c71cb65795f9a73e00000b,Can DNA intercalators function as topoisomeras...


### Split into dataframes [with splitting into sentences] (not finished)

### Train-test splitting

In [12]:
train_question_ids, valid_question_ids = train_test_split(questions_df.index, 
                                                          test_size=0.1,
                                                          random_state=5)
print(f'Train: {len(train_question_ids)}')
print(f'Valid: {len(valid_question_ids)}')

Train: 3636
Valid: 404


In [13]:
# Questions splitting
train_questions_df = questions_df.loc[train_question_ids]
valid_questions_df = questions_df.loc[valid_question_ids]

# Question-Paper matrix splitting
train_question_paper_df = question_paper_df.loc[train_question_ids]
train_question_paper_csr = scipy.sparse.csr_matrix(train_question_paper_df.values, dtype=np.float32)
valid_question_paper_df = question_paper_df.loc[valid_question_ids]
valid_question_paper_csr = scipy.sparse.csr_matrix(valid_question_paper_df.values, dtype=np.float32)

### Tf-idf vectorization

In [14]:
paper_column = 'title_abstract' # title_abstract / title / abstract

# Concatenate all texts
all_texts = np.concatenate([papers_df[paper_column].values, 
                            train_questions_df.question_text.values])

vectorizer = MultipleVectorizer(
    tokenizer=simple_tokenizer,
    lowercase=True,
    min_df=1,
    max_df=1.0,
    do_stemm=False,
    max_features=300_000
)

vectorizer.fit(all_texts)
print(f'Vector size: {len(vectorizer.get_feature_names_out())}')

Fitting vectorizers:


  0%|          | 0/3 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"
100%|██████████| 3/3 [00:43<00:00, 14.49s/it]


Vector size: 434546


In [15]:
train_questions_csr = vectorizer.transform(train_questions_df.question_text.values)
valid_questions_csr = vectorizer.transform(valid_questions_df.question_text.values)
papers_csr = vectorizer.transform(papers_df[paper_column].values)

Transforming:


100%|██████████| 3/3 [00:00<00:00, 10.43it/s]


Transforming:


100%|██████████| 3/3 [00:00<00:00, 79.24it/s]


Transforming:


100%|██████████| 3/3 [00:37<00:00, 12.63s/it]


### Hyperparameter tuning with Bayesian Optimization

In [16]:
from bayes_opt import BayesianOptimization

In [17]:
paper_column = 'title_abstract' # title_abstract / title / abstract

# Concatenate all texts
all_texts = np.concatenate([papers_df[paper_column].values, 
                            train_questions_df.question_text.values])

In [18]:
def black_box_function(min_df,
                       max_df,
                       max_features,
                       threshold, 
                       nr_splits, 
                       Cp, 
                       Cn, 
                       max_iter,
                       eps,
                       bias):
    
    max_features = int(max_features)
    min_df = int(min_df)
    nr_splits = int(nr_splits)
    max_iter = int(max_iter)
    
    # Vectorization
    vectorizer = MultipleVectorizer(
        tokenizer=simple_tokenizer,
        lowercase=True,
        min_df=min_df,
        max_df=max_df,
        do_stemm=False,
        max_features=max_features
    )

    vectorizer.fit(all_texts)
    print(f'Vector size: {len(vectorizer.get_feature_names_out())}')
    train_questions_csr = vectorizer.transform(train_questions_df.question_text.values)
    valid_questions_csr = vectorizer.transform(valid_questions_df.question_text.values)
    papers_csr = vectorizer.transform(papers_df[paper_column].values)
    
    # Train
    hlm_args_dict = {
        'neg_mining_chain': "tfn",
        'model_chain': {
            'threshold': threshold,
            'max_nonzeros_per_label': None,
            'solver_type': "L2R_L2LOSS_SVC_DUAL",
            'Cp': Cp,
            'Cn': Cn,
            'max_iter': max_iter,
            'eps': eps,
            'bias': bias,
            'threads': 8,
            'verbose': 0,
        }
    }
    
    train_params_dict = {
        'mode': "full-model",
        'ranker_level': 1,
        'nr_splits': nr_splits,
        'min_codes': None,
        'shallow': False,
        'rel_mode': "disable",
        'rel_norm': "no-norm",
        'hlm_args': hlm_args_dict
    }
    
    predict_params_dict = {'hlm_args': hlm_args_dict}
    
    train_params = XLinearModel.TrainParams.from_dict(train_params_dict, recursive=True)
    pred_params = XLinearModel.PredParams.from_dict(predict_params_dict, recursive=True)

    pifa_concat = LabelEmbeddingFactory.create(train_question_paper_csr, 
                                           train_questions_csr, 
                                           Z=papers_csr, 
                                           method="pifa_lf_concat")

    cluster_chain = Indexer.gen(pifa_concat, 
                                indexer_type="hierarchicalkmeans",
                                nr_splits=nr_splits)
    
    xlm = XLinearModel.train(train_questions_csr, 
                             train_question_paper_csr, 
                             C=cluster_chain,
                             train_params=train_params)
    
    predicted_question_paper_csr = xlm.predict(valid_questions_csr,
                                               beam_size=10,
                                               only_topk=10)
    
    metric = smat_util.Metrics.generate(valid_question_paper_csr, 
                                        predicted_question_paper_csr, 
                                        topk=10)    
    prec = metric.prec[0] # Take first precision
    
    return prec

In [19]:
pbounds = {
    'min_df':(1, 10),
    'max_df': (0, 1),
    'max_features': (10_000, 500_000),
    'threshold': (0.1, 1),
    'nr_splits': (2, 1024),
    'Cp': (0, 1),
    'Cn': (0, 1),
    'max_iter': (5, 500),
    'eps': (0, 1),
    'bias': (0, 0.5)
}

optimizer = BayesianOptimization(f=black_box_function,
                                 pbounds=pbounds, 
                                 verbose=2,
                                 random_state=5)

In [ ]:
optimizer.maximize(init_points = 10, n_iter = 100)
print("Best result: {}; f(x) = {}.".format(optimizer.max["params"], optimizer.max["target"]))

|   iter    |  target   |    Cn     |    Cp     |   bias    |    eps    |  max_df   | max_fe... | max_iter  |  min_df   | nr_splits | threshold |
-------------------------------------------------------------------------------------------------------------------------------------------------
Fitting vectorizers:


100%|██████████| 3/3 [00:51<00:00, 17.14s/it]


Vector size: 189128
Transforming:


100%|██████████| 3/3 [00:00<00:00, 10.95it/s]


Transforming:


...

100%|██████████| 3/3 [00:00<00:00, 11.38it/s]


Transforming:


100%|██████████| 3/3 [00:00<00:00, 83.39it/s]


Transforming:


100%|██████████| 3/3 [00:38<00:00, 12.71s/it]


### Train

In [16]:
pifa_concat = LabelEmbeddingFactory.create(train_question_paper_csr, 
                                           train_questions_csr, 
                                           Z=papers_csr, 
                                           method="pifa_lf_concat")

In [17]:
cluster_chain = Indexer.gen(pifa_concat, 
                            indexer_type="hierarchicalkmeans",
                            nr_splits=64)

In [18]:
xlm = XLinearModel.train(train_questions_csr, 
                         train_question_paper_csr, 
                         C=cluster_chain)

### Validation

In [19]:
predicted_question_paper_csr = xlm.predict(valid_questions_csr,
                                           beam_size=100,
                                           only_topk=10)

In [20]:
metric = smat_util.Metrics.generate(valid_question_paper_csr, 
                                    predicted_question_paper_csr, 
                                    topk=10)
print(metric)

prec   = 8.42 6.19 5.12 4.46 3.96 3.59 3.22 3.03 2.89 2.82
recall = 3.06 3.78 4.10 4.48 4.64 4.86 4.97 5.13 5.33 5.61


### Checking

In [21]:
def get_papers_by_text(text, 
                       vectorizer,
                       number_of_papers=10, 
                       check_correctness=False):
      
    print('Qestion: ', text)
    vector_text_csr = vectorizer.transform([text])

    papers_csr = xlm.predict(vector_text_csr,
                             beam_size=4,
                             only_topk=number_of_papers)

    prediction = papers_csr.toarray()
    paper_labels = np.nonzero(prediction)[1].tolist()

    # print()
    # print('Found papers: ', len(paper_labels))
    # print(paper_labels)

    paper_logists = np.take(prediction, paper_labels).tolist()

    result = papers_df.loc[paper_labels, ['title', 'abstract', 'snippet_url']]
    result['score'] = paper_logists
    result = result.sort_values('score', ascending=False)

    if check_correctness:
        correct_papers = df.loc[df.question_text == text, 'paper_id'].unique()
        print(f'\nTrue number of papers: {len(correct_papers)}')
        result['correctness'] = result.index.isin(correct_papers)

    return result

In [22]:
correct_examples = [
    'Which genome browser database for DNA shape annotations is available?',
    'Has strimvelis been approved by the European Medicines Agency?',
    'What is the target of Volanesorsen?',
    'Is eptinezumab a small molecule?'
]

interesting_examples = [
    'For which type of diabetes can empagliflozin be used?',
    "Is Semagacestat effective for Alzheimer's Disease?",
    'Which gene is associated with Muenke syndrome?'     
]

In [ ]:
get_papers_by_text('Which gene is associated with Muenke syndrome?',
                   vectorizer=vectorizer, 
                   number_of_papers=30,
                   check_correctness=True)

In [ ]:
# Random example
temp = valid_questions_df.question_text.sample().values[0]
print(temp)

get_papers_by_text(temp,
                   vectorizer=vectorizer,
                   number_of_papers=30,
                   check_correctness=True)

### Highlight answers

In [ ]:
question_text = 'Which genome browser database for DNA shape annotations is available?'

In [ ]:
result = get_papers_by_text(
    question_text,
    vectorizer=vectorizer, 
    number_of_papers=30,
    check_correctness=True
)

In [ ]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained("ozcangundes/T5-base-for-BioQA")
model = T5ForConditionalGeneration.from_pretrained("ozcangundes/T5-base-for-BioQA")

def get_answer(question, context):
    source_encoding=tokenizer(
        question,
        context,
        max_length=512,
        padding="max_length",
        truncation="only_second",
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors="pt")

    generated_ids=model.generate(
        input_ids=source_encoding["input_ids"],
        attention_mask=source_encoding["attention_mask"])

    preds=[tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True) for gen_id in generated_ids]
    
    return "".join(preds)

# To highlight the answers in the results
def highlight_selected_text(row):
    abstract = row["abstract"]
    title = row["title"]
    ext = []
    ext.append(row["abstract_answer"])
    ext.append(row['title_answer'])
    # for k, v in color.items():
    for k in ext:
      if len(k) > 2:
        abstract = abstract.replace(k, f'<span style="color: red; background-color: yellow; font-weight: bold">{k}</span>')
        title = title.replace(k, f'<span style="color: red; background-color: yellow; font-weight: bold">{k}</span>')
    return abstract, title

In [ ]:
result['abstract_answer'] = result['abstract'].apply(lambda x: get_answer(question_text, x))
result['title_answer'] = result['title'].apply(lambda x: get_answer(question_text, x))

In [ ]:
from IPython.core.display import HTML
result['abstract'], result['title'] = zip(*result.apply(highlight_selected_text, axis=1))
display(HTML(result.sample(10).to_html(escape=False)))